
# **Tugas (LSA Topic Modelling)**

## Read Data

In [12]:
# ignore warnings
import warnings
warnings.filterwarnings('ignore')

Data didapatkan dari proses scrapping data youtube. Proses scrapping dapat dilihat pada halaman berikut (scrapping).

In [13]:
import pandas as pd
df=pd.read_csv('https://raw.githubusercontent.com/sholehhuddin/psd/main/datacomment_cnn_fix.csv')
df

,comment,comment (clean)
0,ID\nSkip navigation\nSign in\nPengamat: Suara ...,id skip navigation sign ini pengamat suara pdi...
1,Saya yakin pemilih Djawa Timur adalah pemilih ...,saya yakin pemilih djawa timur adalah pemilih ...
2,"Indonesia butuh pemimpin cerdas berwawasan, pe...",indonesia butuh pemimpin cerdas berwawasan pem...
3,semoga santiasa dalam lindungan allah SWT pak ...,semoga santiasa dalam lindungan allah swt pak ...
4,Ganjar Pranowo presiden 2024 sy non politik 🇮🇩,ganjar pranowo presiden saya non politik bende...
...,...,...
173,sebagian besar masyarakat sdh cerdas mana pemi...,sebagian besar masyarakat sudah cerdas mana pe...
174,Merahkan Indonesia Ganjar Pranowo jadi preside...,merahkan indonesia ganjar pranowo jadi preside...
175,SayA jawa timur siap untuk pak Ganjar....!!!,saya jawa timur siap untuk pak ganjar
176,Intinya itu harus jujur harus adil adu gagasan...,intinya itu harus jujur harus adil adu gagasan...


## Modelling

In [14]:
import nltk
nltk.download('stopwords', quiet=True)

True

In [15]:
from nltk.tokenize import RegexpTokenizer
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer

from nltk.corpus import stopwords
stopwords = stopwords.words('indonesian')

# Membentuk matriks dokumen x kata
tokenizer = RegexpTokenizer(r'\w+')
vectorizer = TfidfVectorizer(lowercase=True,
                        stop_words=stopwords,
                        tokenizer = tokenizer.tokenize)

tfidf_matrix = vectorizer.fit_transform(df['comment (clean)'])

# Melakukan dekomposisi matriks dengan SVD
svd_model = TruncatedSVD(n_components=4)
lsa_matrix = svd_model.fit_transform(tfidf_matrix)

## Bobot kata terhadap masing masing topik

In [16]:
# bobot kata terhadap masing masing topik
terms = vectorizer.get_feature_names_out()

for index, component in enumerate(svd_model.components_):
    zipped = zip(terms, component)
    top_terms_key=sorted(zipped, key = lambda t: t[1], reverse=True)[:2]
    print("Topic "+str(index)+": ",top_terms_key)

Topic 0:  [('ganjar', 0.8523115819958292), ('pranowo', 0.23850534278013902)]
Topic 1:  [('jawa', 0.6628996404993329), ('timur', 0.5119942409557706)]
Topic 2:  [('indonesia', 0.5434383052028479), ('bendera', 0.5312486908886636)]
Topic 3:  [('jatim', 0.4214498646849772), ('cerdas', 0.39085795451748945)]


## Bobot setiap topik terhadap dokumen

In [17]:
# bobot setiap topik terhadap  dokumen
df_lsa = pd.DataFrame(lsa_matrix, columns=["Topik 0", "Topik 1", "Topik 2", "Topik 3"])
df_lsa = pd.concat([df["comment (clean)"], df_lsa], axis=1)
df_lsa['Topik']= df_lsa[['Topik 0', 'Topik 1', 'Topik 2', 'Topik 3']].apply(lambda x: x.argmax(), axis=1)

df_lsa

,comment (clean),Topik 0,Topik 1,Topik 2,Topik 3,Topik
0,id skip navigation sign ini pengamat suara pdi...,0.196849,0.067682,0.099457,0.050521,0
1,saya yakin pemilih djawa timur adalah pemilih ...,0.080015,0.276832,0.061733,0.280286,3
2,indonesia butuh pemimpin cerdas berwawasan pem...,0.070543,0.136089,0.202884,0.215287,3
3,semoga santiasa dalam lindungan allah swt pak ...,0.129100,-0.025708,0.036721,0.042299,0
4,ganjar pranowo presiden saya non politik bende...,0.308477,-0.078390,0.526120,-0.140058,2
...,...,...,...,...,...,...
173,sebagian besar masyarakat sudah cerdas mana pe...,0.040753,0.118836,0.091569,0.220710,3
174,merahkan indonesia ganjar pranowo jadi preside...,0.354313,-0.090007,0.378591,-0.020961,2
175,saya jawa timur siap untuk pak ganjar,0.479174,0.697123,-0.192094,-0.289348,1
176,intinya itu harus jujur harus adil adu gagasan...,0.002090,0.000681,0.005460,0.002163,2


In [18]:
df_lsa['Topik'].value_counts()

0    93
3    34
2    29
1    22
Name: Topik, dtype: int64